# Loop 2 Analysis: Ensemble from Multiple Sources

The evaluator correctly identified that local search on a local optimum is futile.

**Key insight**: The ensemble approach combines the best configurations from multiple sources for each N.

In [1]:
import numpy as np
import pandas as pd
import os
import glob

# Tree polygon vertices
TX = np.array([0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125])
TY = np.array([0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5])

def get_side(xs, ys, degs):
    """Calculate bounding box side length for a configuration."""
    all_px, all_py = [], []
    for x, y, deg in zip(xs, ys, degs):
        rad = np.radians(deg)
        c, s = np.cos(rad), np.sin(rad)
        px = TX * c - TY * s + x
        py = TX * s + TY * c + y
        all_px.extend(px)
        all_py.extend(py)
    return max(max(all_px) - min(all_px), max(all_py) - min(all_py))

def get_score(xs, ys, degs, n):
    """Calculate score for a configuration."""
    side = get_side(xs, ys, degs)
    return side * side / n

def load_submission(filepath):
    """Load a submission and return configs dict."""
    df = pd.read_csv(filepath)
    configs = {}
    for n in range(1, 201):
        group = df[df['id'].str.startswith(f'{n:03d}_')]
        if len(group) == n:
            xs = np.array([float(str(x).lstrip('s')) for x in group['x']])
            ys = np.array([float(str(y).lstrip('s')) for y in group['y']])
            degs = np.array([float(str(d).lstrip('s')) for d in group['deg']])
            configs[n] = {'xs': xs, 'ys': ys, 'degs': degs}
    return configs

print('Functions defined')

Functions defined


In [2]:
# Load all available submissions
submission_files = [
    '/home/code/preoptimized_submission.csv',
    '/home/code/datasets/71.97.csv',
    '/home/code/datasets/72.49.csv',
    '/home/code/datasets/santa-2025.csv',
    '/home/code/datasets/submission.csv',
]

all_submissions = {}
for filepath in submission_files:
    if os.path.exists(filepath):
        name = os.path.basename(filepath)
        configs = load_submission(filepath)
        total_score = sum(get_score(configs[n]['xs'], configs[n]['ys'], configs[n]['degs'], n) for n in configs)
        all_submissions[name] = {'configs': configs, 'total_score': total_score}
        print(f'{name}: {total_score:.6f} (N values: {len(configs)})')
    else:
        print(f'File not found: {filepath}')

preoptimized_submission.csv: 70.743774 (N values: 200)


71.97.csv: 71.972027 (N values: 200)


72.49.csv: 72.495739 (N values: 200)


santa-2025.csv: 70.676102 (N values: 200)


submission.csv: 70.676501 (N values: 200)


In [3]:
# Create ensemble: for each N, pick the best configuration across all sources
ensemble_configs = {}
ensemble_sources = {}

for n in range(1, 201):
    best_score = float('inf')
    best_config = None
    best_source = None
    
    for name, data in all_submissions.items():
        if n in data['configs']:
            cfg = data['configs'][n]
            score = get_score(cfg['xs'], cfg['ys'], cfg['degs'], n)
            if score < best_score:
                best_score = score
                best_config = cfg
                best_source = name
    
    if best_config is not None:
        ensemble_configs[n] = best_config
        ensemble_sources[n] = best_source

# Calculate ensemble total score
ensemble_total = sum(get_score(ensemble_configs[n]['xs'], ensemble_configs[n]['ys'], ensemble_configs[n]['degs'], n) for n in ensemble_configs)
print(f'\nEnsemble total score: {ensemble_total:.6f}')
print(f'Target: 68.922808')
print(f'Gap: {ensemble_total - 68.922808:.6f}')


Ensemble total score: 70.676102
Target: 68.922808
Gap: 1.753294


In [4]:
# Show which source contributed to each N
from collections import Counter
source_counts = Counter(ensemble_sources.values())
print('\nSource contributions to ensemble:')
for source, count in source_counts.most_common():
    print(f'  {source}: {count} N values')


Source contributions to ensemble:
  santa-2025.csv: 197 N values
  preoptimized_submission.csv: 2 N values
  submission.csv: 1 N values


In [5]:
# Show per-N comparison for worst N values
print('\nPer-N comparison (top 20 worst in ensemble):')
scores = []
for n in range(1, 201):
    cfg = ensemble_configs[n]
    score = get_score(cfg['xs'], cfg['ys'], cfg['degs'], n)
    scores.append((n, score, ensemble_sources[n]))

scores.sort(key=lambda x: -x[1])
for n, score, source in scores[:20]:
    print(f'  N={n:3d}: {score:.6f} (from {source})')


Per-N comparison (top 20 worst in ensemble):
  N=  1: 0.661250 (from preoptimized_submission.csv)
  N=  2: 0.450779 (from submission.csv)
  N=  3: 0.434745 (from preoptimized_submission.csv)
  N=  5: 0.416850 (from santa-2025.csv)
  N=  4: 0.416545 (from santa-2025.csv)
  N=  7: 0.399897 (from santa-2025.csv)
  N=  6: 0.399610 (from santa-2025.csv)
  N=  9: 0.387415 (from santa-2025.csv)
  N=  8: 0.385407 (from santa-2025.csv)
  N= 15: 0.379203 (from santa-2025.csv)
  N= 10: 0.376630 (from santa-2025.csv)
  N= 21: 0.376451 (from santa-2025.csv)
  N= 20: 0.376057 (from santa-2025.csv)
  N= 11: 0.375736 (from santa-2025.csv)
  N= 22: 0.375258 (from santa-2025.csv)
  N= 16: 0.374128 (from santa-2025.csv)
  N= 26: 0.373997 (from santa-2025.csv)
  N= 12: 0.372724 (from santa-2025.csv)
  N= 13: 0.372323 (from santa-2025.csv)
  N= 25: 0.372144 (from santa-2025.csv)


In [6]:
# Compare individual N values between sources
print('\nDetailed comparison for worst N values:')
for n, _, _ in scores[:10]:
    print(f'\nN={n}:')
    for name, data in all_submissions.items():
        if n in data['configs']:
            cfg = data['configs'][n]
            score = get_score(cfg['xs'], cfg['ys'], cfg['degs'], n)
            marker = ' <-- BEST' if name == ensemble_sources[n] else ''
            print(f'  {name}: {score:.6f}{marker}')


Detailed comparison for worst N values:

N=1:
  preoptimized_submission.csv: 0.661250 <-- BEST
  71.97.csv: 0.661250
  72.49.csv: 0.661250
  santa-2025.csv: 0.661250
  submission.csv: 0.661250

N=2:
  preoptimized_submission.csv: 0.450779
  71.97.csv: 0.450779
  72.49.csv: 0.450779
  santa-2025.csv: 0.450779
  submission.csv: 0.450779 <-- BEST

N=3:
  preoptimized_submission.csv: 0.434745 <-- BEST
  71.97.csv: 0.434745
  72.49.csv: 0.434745
  santa-2025.csv: 0.434745
  submission.csv: 0.434745

N=5:
  preoptimized_submission.csv: 0.416850
  71.97.csv: 0.417047
  72.49.csv: 0.417047
  santa-2025.csv: 0.416850 <-- BEST
  submission.csv: 0.416850

N=4:
  preoptimized_submission.csv: 0.416545
  71.97.csv: 0.416589
  72.49.csv: 0.416589
  santa-2025.csv: 0.416545 <-- BEST
  submission.csv: 0.416545

N=7:
  preoptimized_submission.csv: 0.399897
  71.97.csv: 0.400195
  72.49.csv: 0.400207
  santa-2025.csv: 0.399897 <-- BEST
  submission.csv: 0.399897

N=6:
  preoptimized_submission.csv: 0.39

In [7]:
# Generate ensemble submission
rows = []
for n in range(1, 201):
    cfg = ensemble_configs[n]
    for i in range(n):
        rows.append({
            'id': f'{n:03d}_{i}',
            'x': f's{cfg["xs"][i]}',
            'y': f's{cfg["ys"][i]}',
            'deg': f's{cfg["degs"][i]}'
        })

df_out = pd.DataFrame(rows)
df_out.to_csv('/home/submission/submission.csv', index=False)
print(f'Saved ensemble submission with score {ensemble_total:.6f}')
print(f'Total rows: {len(df_out)}')

Saved ensemble submission with score 70.676102
Total rows: 20100


In [8]:
# Let's check if there are better solutions by downloading more kernel outputs
# The bbox3-ensemble-update kernel uses many kernel outputs as inputs

# Check what kernel outputs are available
import subprocess
result = subprocess.run(['kaggle', 'kernels', 'output', 'saspav/santa-submission', '-p', '/home/code/datasets/saspav_output'], 
                       capture_output=True, text=True)
print(result.stdout)
print(result.stderr)

In [9]:
# Load the jazivxt/why-not kernel output and check its score
jazivxt_configs = load_submission('/home/code/datasets/jazivxt_output/submission.csv')
jazivxt_score = sum(get_score(jazivxt_configs[n]['xs'], jazivxt_configs[n]['ys'], jazivxt_configs[n]['degs'], n) for n in jazivxt_configs)
print(f'jazivxt/why-not output score: {jazivxt_score:.6f}')

jazivxt/why-not output score: 70.676501


In [10]:
# Load the eazy-optimizer kernel output and check its score
eazy_configs = load_submission('/home/code/datasets/eazy_output/submission.csv')
eazy_score = sum(get_score(eazy_configs[n]['xs'], eazy_configs[n]['ys'], eazy_configs[n]['degs'], n) for n in eazy_configs)
print(f'eazy-optimizer output score: {eazy_score:.6f}')

eazy-optimizer output score: 70.699761


In [12]:
# Load the ashraful kernel output and check its score
ashraful_configs = load_submission('/home/code/datasets/ashraful_output/submission.csv')
ashraful_score = sum(get_score(ashraful_configs[n]['xs'], ashraful_configs[n]['ys'], ashraful_configs[n]['degs'], n) for n in ashraful_configs)
print(f'ashraful output score: {ashraful_score:.6f}')

ashraful output score: 70.734281


In [13]:
# Create comprehensive ensemble from all available sources
all_sources = {
    'preoptimized': load_submission('/home/code/preoptimized_submission.csv'),
    '71.97': load_submission('/home/code/datasets/71.97.csv'),
    '72.49': load_submission('/home/code/datasets/72.49.csv'),
    'santa-2025': load_submission('/home/code/datasets/santa-2025.csv'),
    'telegram': load_submission('/home/code/datasets/submission.csv'),
    'jazivxt': load_submission('/home/code/datasets/jazivxt_output/submission.csv'),
    'eazy': load_submission('/home/code/datasets/eazy_output/submission.csv'),
    'ashraful': load_submission('/home/code/datasets/ashraful_output/submission.csv'),
}

# Print scores for each source
print('Source scores:')
for name, configs in all_sources.items():
    total = sum(get_score(configs[n]['xs'], configs[n]['ys'], configs[n]['degs'], n) for n in configs)
    print(f'  {name}: {total:.6f}')

Source scores:
  preoptimized: 70.743774


  71.97: 71.972027
  72.49: 72.495739


  santa-2025: 70.676102
  telegram: 70.676501


  jazivxt: 70.676501
  eazy: 70.699761


  ashraful: 70.734281


In [14]:
# Create ensemble: for each N, pick the best configuration across all sources
ensemble_configs = {}
ensemble_sources = {}

for n in range(1, 201):
    best_score = float('inf')
    best_config = None
    best_source = None
    
    for name, configs in all_sources.items():
        if n in configs:
            cfg = configs[n]
            score = get_score(cfg['xs'], cfg['ys'], cfg['degs'], n)
            if score < best_score:
                best_score = score
                best_config = cfg
                best_source = name
    
    if best_config is not None:
        ensemble_configs[n] = best_config
        ensemble_sources[n] = best_source

# Calculate ensemble total score
ensemble_total = sum(get_score(ensemble_configs[n]['xs'], ensemble_configs[n]['ys'], ensemble_configs[n]['degs'], n) for n in ensemble_configs)
print(f'\\nEnsemble total score: {ensemble_total:.6f}')
print(f'Target: 68.922808')
print(f'Gap: {ensemble_total - 68.922808:.6f}')

# Show source contributions
from collections import Counter
source_counts = Counter(ensemble_sources.values())
print('\\nSource contributions to ensemble:')
for source, count in source_counts.most_common():
    print(f'  {source}: {count} N values')

\nEnsemble total score: 70.676092
Target: 68.922808
Gap: 1.753284
\nSource contributions to ensemble:
  santa-2025: 191 N values
  eazy: 5 N values
  preoptimized: 2 N values
  telegram: 1 N values
  jazivxt: 1 N values
